# Notebook to test the LSI implementation

In [ ]:
using BenchmarkTools
using PyPlot

In [ ]:
cd("../")
# Let us observe the progress during the RT computations
ENV["XRTM_PROGRESS"] = "1"

# For good font rendering, we suggest installing the "JuliaMono" font
# which provides full support for the various Unicode glyphs.
ENV["GKS_FONTPATH"] = "~/Library/Fonts/"

In [ ]:
Nhigh = 16

hmo1 = Dict()
hmo1["solvers"] = ["single"]
hmo1["add"] = true
hmo1["streams"] = Nhigh
hmo1["options"] = [
    "output_at_levels",
    "calc_derivs",
    "source_solar",
    "vector",
    "psa",
    "sfi",
    ]

#push!(hmo1["options"], "delta_m")
#push!(hmo1["options"], "n_t_tms")


hmo2 = Dict()
hmo2["solvers"] = ["two_os"]
hmo2["add"] = true
hmo2["streams"] = Nhigh
hmo2["options"] = [
    "output_at_levels",
    "calc_derivs",
    "source_solar",
    "vector",
    "psa",
    "sfi",
]

hmo3 = Dict()
hmo3["solvers"] = ["eig_bvp"]
hmo3["add"] = true
hmo3["streams"] = Nhigh
hmo3["options"] = [
    "output_at_levels",
    "calc_derivs",
    "source_solar",
    "psa",
    "sfi",
    ]

#push!(hmo3["options"], "delta_m")
#push!(hmo3["options"], "n_t_tms")

high_options = [
    hmo1,
    hmo2,
    hmo3
]


In [ ]:
# Define command-line arguments
my_args = [
    "--solar_model",  "./example_data/l2_solar_model.h5", # Path to the solar model file
    "--L1b", "./example_data/2021030111564431_inputs.h5", # Path to the L1B data location
    "--L2Met", "./example_data/2021030111564431_inputs.h5", # Path to the L2Met data location
    "--L2CPr", "./example_data/2021030111564431_inputs.h5", # Path to the L2CPr data location
    #######################################################
    "--sounding_id", "2021030111564431", # Sounding ID to retrieve
    "--spec", "1", # Which spectra to retrieve? 1 = O2-A, 2 = Weak CO2 (1.6 µm), 3 = Strong CO2 (2.06 µm)
    "--aerosols", "true", # Have aerosols in the atmopshere?
    "--retrieve_aerosols", "false", # Retrieve aerosol parameters? (AOD, height, width)
    "--LSI", "false",
    "--o2_scale", "1.0048", # Spectroscopy scaling factor for Oxygen
    "--co2_scale_weak", "0.994", # Spectroscopy scaling factor for CO2 for the weak CO2 band
    "--co2_scale_strong", "0.998", # Spectroscopy scaling factor for CO2 for the strong CO2 band
    "--gamma", "1000.0", # Levenberg-Marquardt γ parameter (to avoid diverging steps, we make this large)
    "--dsigma_scale", "2.0", # dσ^2 ~ a parameter that controls convergence
    "--max_iterations", "0", # Number of maximal iterations
    "--output", "2021030111564431.h5", # Name of the output file
]

# Get rid of existing command line arguments
empty!(ARGS)

# Push them in
for a in my_args
    push!(ARGS, a)
end

buf, solver, fm_kwargs = include("./run.jl");

In [ ]:
swin = buf.spectral_window[1]
rt = buf.rt[swin]
low_options = deepcopy(rt.model_options)

In [ ]:
calculate_earth_optical_properties!(
    rt.optical_properties,
    rt.scene,
    solver.state_vector
    )

In [ ]:
LSI_bounds = [
            0, 0.001, 0.01, 0.05, 0.1, 0.15, 0.225, 0.338, 0.508,
            0.7, 0.85, 1, 1.15, 1.4, 1.719, 2.581, 3.875, 5.817,
            8.733, 13, 20, 30, 44, 70, 100, 200, 300, 1e+06
        ];

In [ ]:
length(LSI_bounds) - 1

In [ ]:
empty!(rt.model_options);
for mo in high_options
    push!(rt.model_options, mo)
end

In [ ]:
# Calculate high-res run
@time solver.forward_model(solver.state_vector; fm_kwargs...)

In [ ]:
full_rt = deepcopy(rt.hires_radiance);

In [ ]:
empty!(rt.model_options);
mo1 = Dict()
mo2 = Dict()

mo1["solvers"] = ["single"]
mo1["add"] = true
mo1["streams"] = 2
mo1["options"] = [
    "output_at_levels",
    #"calc_derivs",
    "source_solar",
    "vector",
    "psa",
    "sfi",
    ]


mo2["solvers"] = ["two_stream"]
mo2["add"] = true
mo2["streams"] = 2
mo2["options"] = [
    "output_at_levels",
    #"calc_derivs",
    "source_solar",
    "psa",
    "sfi"
    ]

push!(rt.model_options, mo1)
push!(rt.model_options, mo2)

In [ ]:
@time solver.forward_model(solver.state_vector; fm_kwargs...)
low_rt = deepcopy(rt.hires_radiance);

In [ ]:
begin 
    @time lsi = LSIRTMethod(
        LSI_bounds,
        rt,
        high_options
    )
    @time perform_LSI_correction!(lsi)
end

In [ ]:
plt.plot(full_rt.S[:,1]);
plt.plot(low_rt.S[:,1]);
plt.plot(rt.hires_radiance.S[:,1], "k--")

In [ ]:
plt.plot(rt.hires_radiance.S[:,1] .- full_rt.S[:,1])

In [ ]:
lsi.bin_rad_lo[1,1]

In [ ]:
lsi.bin_edge_rad_lo[1,1] .* 20000

In [ ]:
calculate_binned_properties!(lsi, 1, 1, false);
lsi.RT_bin.optical_properties.total_tau[1,:]

In [ ]:
calculate_binned_properties!(lsi, 1, 1, true);
clear!(lsi.RT_bin_edge)
for mo in high_options
    RE._calculate_radiances_and_wfs_XRTM!(lsi.RT_bin_edge, lsi.RT_bin_edge.scene.observer, mo)
end
lsi.RT_bin_edge.hires_radiance

In [ ]:
calculate_binned_properties!(lsi, 1, 1, false);
clear!(lsi.RT_bin)
for mo in high_options
    RE._calculate_radiances_and_wfs_XRTM!(lsi.RT_bin, lsi.RT_bin.scene.observer, mo)
end
lsi.RT_bin.hires_radiance

In [ ]:
lsi.RT_bin_edge.optical_properties.spectral_window.ww_grid

In [ ]:
lsi.RT_bin.optical_properties.spectral_window.ww_grid